This notebook just tests HuggingFace models

In [18]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from scipy.special import softmax
import urllib.request
import csv

# Define model
MODEL = "cardiffnlp/twitter-roberta-base-sentiment"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# Download label mapping (negative, neutral, positive)
labels = []
mapping_link = "https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
    labels = [row[1] for row in csvreader if len(row) > 1]

def classify_tweet(text):
    # Tokenize text
    encoded_input = tokenizer(text, return_tensors="pt")
    
    # Run model and get logits
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    
    # Apply softmax to convert to probabilities
    scores = softmax(scores)
    
    # Rank labels by confidence
    ranking = np.argsort(scores)[::-1]
    
    # Get highest-ranked sentiment
    sentiment = labels[ranking[0]]
    confidence = scores[ranking[0]]
    
    return {"label": sentiment, "score": round(float(confidence), 4)}

# Test cases to classify
test_cases = [
    "This movie was lit 🔥🔥!",  # positive
    "I love this song so much 😍",  # positive
    "I’m having a great time, everyone is so friendly!",  # positive
    "Such a great experience, I highly recommend it!",  # positive
    
    "Ugh, I hate waiting for this!",  # negative
    "Why is this place always so crowded? 😩",  # negative
    "I can't believe how bad the service was!",  # negative
    "This place sucks!", # negative
    
    "It's just okay, not great but not terrible.",  # neutral
    "I don't know what to feel about it. 🤔",  # neutral
    "The food was terrible, but the music was good!",  # neutral
    "I'm not sure" # neutral
]

# Classify each test case and print the result
for i, text in enumerate(test_cases, 1):
    result = classify_tweet(text)
    print(f"Test Case {i}: {text} Classification: {result}")


Test Case 1: This movie was lit 🔥🔥! Classification: {'label': 'positive', 'score': 0.9712}
Test Case 2: I love this song so much 😍 Classification: {'label': 'positive', 'score': 0.9895}
Test Case 3: I’m having a great time, everyone is so friendly! Classification: {'label': 'positive', 'score': 0.9914}
Test Case 4: Such a great experience, I highly recommend it! Classification: {'label': 'positive', 'score': 0.9904}
Test Case 5: Ugh, I hate waiting for this! Classification: {'label': 'negative', 'score': 0.9745}
Test Case 6: Why is this place always so crowded? 😩 Classification: {'label': 'negative', 'score': 0.9447}
Test Case 7: I can't believe how bad the service was! Classification: {'label': 'negative', 'score': 0.974}
Test Case 8: This place sucks! Classification: {'label': 'negative', 'score': 0.9815}
Test Case 9: It's just okay, not great but not terrible. Classification: {'label': 'positive', 'score': 0.7314}
Test Case 10: I don't know what to feel about it. 🤔 Classification: {

In [20]:
from transformers import pipeline

# Define model and load sentiment pipeline for Yelp reviews
model_id = "nlptown/bert-base-multilingual-uncased-sentiment"
yelp_pipeline = pipeline("sentiment-analysis", model=model_id)

# Test cases to classify
test_cases = [
    "The food was fantastic, but the service was slow.",  # mixed sentiment
    "This place was terrible. The food was cold and the staff was rude.",  # negative
    "I had a wonderful experience here! The staff was friendly, and the food was amazing!",  # positive
    "The ambiance was nice, but the food wasn't great. I expected better.",  # neutral
    "Absolutely awful. Never coming back.",  # negative
    "Loved the decor and the staff, but the food was too spicy for me.",  # mixed sentiment
    "This restaurant is incredible! One of the best meals I've ever had!",  # positive
    "Not worth the price. Very disappointing.",  # negative
    "Great food, great service. Highly recommend!",  # positive
    "The waiter was nice, but the food was just okay.",  # neutral
]

# Classify each Yelp review and print the result
for i, text in enumerate(test_cases, 1):
    result = yelp_pipeline(text)
    print(f"Test Case {i}: {text} Classification: {result}")


Device set to use cpu


Test Case 1: The food was fantastic, but the service was slow. Classification: [{'label': '3 stars', 'score': 0.49869871139526367}]
Test Case 2: This place was terrible. The food was cold and the staff was rude. Classification: [{'label': '1 star', 'score': 0.8552736043930054}]
Test Case 3: I had a wonderful experience here! The staff was friendly, and the food was amazing! Classification: [{'label': '5 stars', 'score': 0.8434882760047913}]
Test Case 4: The ambiance was nice, but the food wasn't great. I expected better. Classification: [{'label': '3 stars', 'score': 0.6580864787101746}]
Test Case 5: Absolutely awful. Never coming back. Classification: [{'label': '1 star', 'score': 0.9784294366836548}]
Test Case 6: Loved the decor and the staff, but the food was too spicy for me. Classification: [{'label': '3 stars', 'score': 0.6273780465126038}]
Test Case 7: This restaurant is incredible! One of the best meals I've ever had! Classification: [{'label': '5 stars', 'score': 0.98102611303